# SD-TSIA211 : RECOMMENDATION

## Remi Jaylet

### imports

In [13]:
from movielens_utils import load_movielens, objective
import numpy as np
from scipy.sparse.linalg import svds
from scipy.optimize import check_grad

### 1 - Presentation of the models

In [8]:
R, mask = load_movielens("ml-100k/u.data", minidata=False)
print("R = ", R)
print("mask = ", mask)
print("dimensions = (", len(R), ",", len(R[0]), ")")
print("number of ratings :", sum(sum(mask)))

R =  [[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]
mask =  [[ True  True  True ... False False False]
 [ True False False ... False False False]
 [False False False ... False False False]
 ...
 [ True False False ... False False False]
 [False False False ... False False False]
 [False  True False ... False False False]]
dimensions = ( 943 , 1682 )
number of ratings : 100000


In [9]:
R, mask = load_movielens("ml-100k/u.data", minidata=True)
print("dimensions = (", len(R), ",", len(R[0]), ")")

dimensions = ( 100 , 200 )


When the minidata option is activated, the function only returns a small portion of the R matrix of only 100 users over 200 films.

The dimensions of the matric suggest that are ratings from 943 users on 1682 movies.
By calculation the number of True values in the mask, we can we that there is 100000 ratings ranging from 0 to 5.

### 2 - Find P when $ Q^0 $ is fixed

In [21]:
# initialize variables
R, mask = load_movielens("ml-100k/u.data")
Q0, s, P0 = svds(R, k=4)
rho = 0.3

# define g and gradient of g (use ravel and reshape to be able to check the value of the gradient)
def g(P):
    P = np.reshape(P, (4,1682))
    tmp = (R - Q0.dot(P)) * mask
    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))
    return val

def grad_g(P):
    P = np.reshape(P, (4,1682))
    tmp = (R - Q0.dot(P)) * mask
    grad_P = (Q0.transpose()).dot(tmp) + rho*P
    return np.ravel(grad_P)

In [ ]:
error = check_grad(g, grad_g, x0=np.ravel(P0))
print("error on the gradient :", error)

In [ ]:
def gradient(g, P0, gamma, epsilon):
    val, grad_P = objective(P0, Q0, R, mask, rho)
    while np.linalg.norm(grad_P).any() > epsilon:
        P0 = P0 - gamma*grad_P
        grad_P = objective(P0, Q0, R, mask, rho)[1]
    val = objective(P0, Q0, R, mask, rho)[0]
    return val, P0

In [ ]:
gradient(g, P0, 1, 1)

In [ ]:
def gradient_2(g, P0, epsilon):
    val, grad_P = objective(P0, Q0, R, mask, rho)
    while np.linalg.norm(grad_P).any() > epsilon:
        gamma = scipy.optimize.linesearch(g, grad_g, np.ravel(P0), -grad) # use line search
        P0 = P0 - gamma*grad_P
        grad_P = objective(P0, Q0, R, mask, rho)[1]
    val = objective(P0, Q0, R, mask, rho)[0]
    return val, P0

In [ ]:
gradient_2(g, P0, 1)

### 3 - Resolution of the full problem

1471.5465355048495


In [29]:
def gradient(g, P0, gamma, epsilon):
    val, grad_P = objective(P0, Q0, R, mask, rho)
    while np.linalg.norm(grad_P).any() > epsilon:
        P0 = P0 - gamma*grad_P
        grad_P = objective(P0, Q0, R, mask, rho)[1]
    val = objective(P0, Q0, R, mask, rho)[0]
    return val, P0

In [30]:
gradient(g, P0, 1, 1)

(685091.7826478565,
 array([[-1.62060750e-02, -3.23391424e-03, -4.88005961e-02, ...,
          9.53260775e-04, -7.73342642e-05, -1.74971250e-03],
        [-1.69737618e-02, -6.25039193e-02, -1.16405039e-02, ...,
          5.33024145e-04, -4.54336533e-04, -2.61400068e-04],
        [ 8.72397853e-02,  7.02505798e-03,  2.86181725e-02, ...,
          4.48134760e-04, -1.05231342e-04, -2.03151884e-04],
        [ 9.59509371e-02,  3.51795155e-02,  1.99288117e-02, ...,
          3.03747116e-05,  3.31055915e-04,  3.16852950e-04]]))

In [32]:
def gradient_2(g, P0, epsilon):
    val, grad_P = objective(P0, Q0, R, mask, rho)
    while np.linalg.norm(grad_P).any() > epsilon:
        gamma = scipy.optimize.linesearch(g, grad_g, np.ravel(P0), -grad) # use line search
        P0 = P0 - gamma*grad_P
        grad_P = objective(P0, Q0, R, mask, rho)[1]
    val = objective(P0, Q0, R, mask, rho)[0]
    return val, P0

In [34]:
gradient_2(g, P0, 1)

(685091.7826478565,
 array([[-1.62060750e-02, -3.23391424e-03, -4.88005961e-02, ...,
          9.53260775e-04, -7.73342642e-05, -1.74971250e-03],
        [-1.69737618e-02, -6.25039193e-02, -1.16405039e-02, ...,
          5.33024145e-04, -4.54336533e-04, -2.61400068e-04],
        [ 8.72397853e-02,  7.02505798e-03,  2.86181725e-02, ...,
          4.48134760e-04, -1.05231342e-04, -2.03151884e-04],
        [ 9.59509371e-02,  3.51795155e-02,  1.99288117e-02, ...,
          3.03747116e-05,  3.31055915e-04,  3.16852950e-04]]))

### 3 - Resolution of the full problem